# AgentsVille Trip Planner - Project Assignment

In this project, you'll implement an AI system to help you plan a trip--to the wonderful city of AgentsVille!

Your AI system will demonstrate advanced LLM reasoning techniques including:

1. **Role-Based Prompting** - Your agent will act as a specialized travel planner
2. **Chain-of-Thought Reasoning** - Step-by-step planning of itineraries
3. **ReAct Prompting** - Thought → Action → Observation cycles
4. **Feedback Loops** - Self-evaluation using tools in the ReAct loop to find mistakes and improve plans

You'll simulate external API calls to gather weather data and activities. Then, process this information to create personalized travel itineraries based on interests and other constraints. Last, you'll implement a feedback loop to refine the plan.

Your task is to build a travel agent that can plan the perfect AgentsVille vacation!

## Project Instructions

Here are the steps you'll follow:

1. **Define Vacation Details**
    - Specify the trip duration, interests, and constraints.
    - Use Pydantic to structure and verify this information in a class called `VacationInfo`.
2. **Review Weather and Activity schedules**
    - Simulate API calls to gather weather data and available activities in bulk
    - Review the data manually to understand the available options
3. **The ItineraryAgent**
    - Implement an agent that generates a day-by-day itinerary based on the vacation details
    - The system prompt will guide the agent's reasoning through a step-by-step planning process to take travel preferences (e.g. destination, dates, interests) and generate a detailed day-by-day itinerary
    - Craft the components of the prompt (including the role, task/instructions, output format, examples, and context) to elicit the best possible itinerary in one LLM call
4. **Evaluating the Itinerary**
    - Evaluate the itinerary using a set of criteria to ensure a high-quality travel plan
        - For instance, does the itinerary match the city and the dates requested?
        - Or, is the total cost calulation accurate and is it within budget?
        - Or, does the agent hallucinate any activities that are not available?
        - Or, does the agent suggest activities that are not suitable for the weather? This specific evaluation function will require the use of an LLM to compare the event description against the weather data.
5. **Defining the Tools**
    - We will define four tools to assist the agent
        - **calculator_tool**: to accurately calculate costs
        - **get_activities_by_date_tool**: to retrieve activities for a specific date
        - **run_evals_tool**: to evaluate the itinerary against the criteria
        - **final_answer_tool**: to provide the final answer in a structured format
6. **The ItineraryRevisionAgent**
    - We will implement a second agent that revises the itinerary based on feedback using the ReAct THOUGHT → ACTION → OBSERVATION cycle.
        - The LLM will first generated a THOUGHT / ACTION message, which contains reasoning steps and a tool call invocation.
        - The Python code will parse the tool call and execute it, returning the result as a string to the LLM in an OBSERVATION message.
        - After this cycle repeats n number of times, the LLM will invoke the final_answer_tool to signal to the Python code to end the loop and return the final answer.
    - This agent will also **incorporate feedback on the initial itinerary** from the travelers to ensure the final plan has **at least 2 activities per day**. A new evaluation function using a powerful LLM will be created to check this user feedback.
    - The agent will use the tools above to refine the plan iteratively, checking the weather and available activities, and ensuring the itinerary meets all constraints.
7. **Something just for fun!**
    - To wrap things up we'll create a fun, narrative summary of the trip, highlighting the best activities and experiences!

## Initial Setup

Let's start with settin up our environment and defining the vacation details.

In [1]:
# When using VSCode in the Udacity workspace, add /workspace to the PYTHON_PATH
import os
import sys

WORKSPACE_DIRECTORY = "/workspace"
if os.path.exists(WORKSPACE_DIRECTORY) and WORKSPACE_DIRECTORY not in sys.path:
    sys.path.append(WORKSPACE_DIRECTORY)
    print(f"Added {WORKSPACE_DIRECTORY} to the Python path")

In [2]:
# Install required packages if not already installed
# No changes needed here.
%pip install -q json-repair==0.47.1 numexpr==2.11.0 openai==1.74.0 pandas==2.3.0 pydantic==2.11.7 python-dotenv==1.1.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# If using the Vocareum API endpoint
# TODO: Fill in the missing parts marked with **********
from openai import OpenAI

client = OpenAI(
    # Change the base_url when using the Vocareum API endpoint
    # If using the OpenAI API endpoint, you can comment out the base_url line
    base_url="https://openai.vocareum.com/v1",
    # Uncomment one of the following
    api_key="XXX"  # <--- TODO: Fill in your Vocareum API key here
)


In [4]:
# Throughout this project you can experiment with different OpenAI models.
# By default we will use GPT-4.1-mini, which is a good balance of speed and cost.
from enum import Enum

class OpenAIModel(str, Enum):
    GPT_41 = "gpt-4.1"  # Strong default choice for development tasks, particularly those requiring speed, responsiveness, and general-purpose reasoning. 
    GPT_41_MINI = "gpt-4.1-mini"  # Fast and affordable, good for brainstorming, drafting, and tasks that don't require the full power of GPT-4.1.
    GPT_41_NANO = "gpt-4.1-nano"  # The fastest and cheapest model, suitable for lightweight tasks, high-frequency usage, and edge computing.

MODEL = OpenAIModel.GPT_41_NANO # Default model for this project


## Define Vacation Details

Let's encode the details of our vacation in JSON format and verify it using Pydantic.

In practice, a chatbot agent could gather the information of a user. After it has gathered all the information it needs, it could generate this JSON object from the chat transcript. We will skip that step to focus on the itinerary generation itself.

In [5]:
# The Vacation Info Data Structure
# No changes needed here, but you may choose to personalize the data.

VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["tennis", "cooking", "comedy", "technology"],
        },
        {
            "name": "Hiro",
            "age": 25,
            # Possible interests: art, cooking, comedy, dancing, fitness, gardening, hiking, movies,
            # music, photography, reading, sports, technology, theatre, tennis, writing
            "interests": ["reading", "music", "theatre", "art"],
        },
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",  # Mock data exists for 2025-06-10
    "date_of_departure": "2025-06-12",  # ...until 2025-06-15.
    "budget": 130,  # Budget is in fictional currency units.
}

In [6]:
# Validate the data structure using Pydantic
# TODO: Fill in the missing parts marked with **********

from project_lib import Interest
from typing import List
from pydantic import BaseModel
import datetime
from pprint import pprint

class Traveler(BaseModel):
    """A traveler with a name, age, and list of interests.
    
    Attributes:
        name (str): The name of the traveler.
        age (int): The age of the traveler.
        interests (List[Interest]): A list of interests of the traveler.
    """
    name: str
    age: int
    interests: List[Interest]

class VacationInfo(BaseModel):
    """Vacation information including travelers, destination, dates, and budget.
    Attributes:
        travelers (List[Traveler]): A list of travelers.
        destination (str): The vacation destination.
        date_of_arrival (datetime.date): The date of arrival.
        date_of_departure (datetime.date): The date of departure.
        budget (int): The budget for the vacation in fictional currency units.
    """
    travelers: List[Traveler]
    destination: str
    date_of_arrival: datetime.date
    date_of_departure: datetime.date
    budget: int


# Validate the VacationInfo data structure
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

{'budget': 130,
 'date_of_arrival': datetime.date(2025, 6, 10),
 'date_of_departure': datetime.date(2025, 6, 12),
 'destination': 'AgentsVille',
 'travelers': [{'age': 30,
                'interests': [tennis, cooking, comedy, technology],
                'name': 'Yuri'},
               {'age': 25,
                'interests': [reading, music, theatre, art],
                'name': 'Hiro'}]}
✅ VacationInfo data structure is valid!


## Review Weather and Activity Schedules

Now that we have the trip details, we can retrieve the weather and activity schedules for the dates of the trip.

We will call an API to get all the data at once, in order to be able to include it in the context for our itinerary planning agent.

Also, we will format the data as Pandas DataFrames for easier viewing. Take the time to read and understand the data to see if the agent notices the same things you do. For instance:
- What does the weather look like for the trip? On what days it is sunny, rainy, or cloudy?
- What activities are available on each day? Are there any special events or festivals related to the user's interests?

In [7]:
# The `call_weather_api_mocked` mocks calling a weather API to get weather data
# TODO: Fill in the missing parts marked with **********

from project_lib import call_weather_api_mocked
import pandas as pd

pd.set_option("display.max_colwidth", None)  # Show full content in DataFrame cells

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        start=pd.to_datetime(vacation_info.date_of_arrival),   # e.g., datetime.date or datetime.datetime
        end=pd.to_datetime(vacation_info.date_of_departure),       # inclusive end
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

weather_for_dates_df

,date,city,temperature,temperature_unit,condition,description
0,2025-06-10,AgentsVille,31,celsius,clear,A bright and sunny day in AgentsVille with clear skies and warm temperatures. Perfect weather for outdoor activities!
1,2025-06-11,AgentsVille,34,celsius,partly cloudy,"A warm day with periods of sunshine and mixed clouds, making it a perfect opportunity for outdoor activities."
2,2025-06-12,AgentsVille,28,celsius,thunderstorm,"A thunderstorm is expected to roll in during the afternoon, bringing heavy rain and gusty winds. The atmosphere will feel charged with humidity, creating a sultry and dramatic setting as clouds build in the sky."


In [8]:
# The `call_activities_api_mocked` function returns the activities for a given date and city.
# TODO: Fill in the missing parts marked with **********

from project_lib import call_activities_api_mocked

activities_for_dates = [
    activity
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        start=vacation_info.date_of_arrival,
        end=vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)

activities_for_dates_df

,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00,2025-06-10 11:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-1,Serve & Savor: Tennis and Taste Luncheon,2025-06-10 12:00,2025-06-10 13:30,"The Grand Racquet Terrace, AgentsVille","Join us for 'Serve & Savor,' the ultimate crossover event for cooking and tennis enthusiasts in AgentsVille! Kick off your lunch hour with a friendly round of doubles on our outdoor courts, then unwind with a hands-on cooking workshop led by a local chef, where you'll prepare and enjoy delicious energy-boosting recipes. Whether you come for the sport or the flavors, this energizing luncheon celebrates both passions in a lively outdoor setting. Ideal for anyone who loves to play, cook, or simply savor fresh food and fun!",20,"[cooking, tennis]"
2,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00,2025-06-10 17:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"
3,event-2025-06-10-3,AgentsVille Twilight Writing Escape,2025-06-10 19:00,2025-06-10 21:00,"The Ink Loft, 12 Quill Lane, AgentsVille","Join fellow writers for an inspiring evening at The Ink Loft, where words flow as freely as the coffee! This writing-themed event welcomes all—novelists, poets, bloggers, or anyone with a passion for storytelling. Set indoors in AgentsVille's coziest lounge, enjoy writing games, group prompts, and opportunities to read your work aloud. Connect, create, and celebrate the art of writing in this creative indoor haven.",15,"[writing, reading, art]"
4,event-2025-06-11-0,Morning Groove Dance Party,2025-06-11 09:00,2025-06-11 10:30,"Rhythm Hall, Center Plaza, AgentsVille","Start your day with energy and joy at the Morning Groove Dance Party! This lively event welcomes dancers of all levels to join a vibrant indoor session filled with upbeat music and fun routines. Whether you love modern pop, Latin beats, or classic disco, our dance instructors will guide you to move and groove. Connect with fellow dance lovers in the colorful atmosphere of Rhythm Hall. Perfect for fans of dancing, music, and fitness. Let the rhythm move you! (Indoor event.)",15,"[dancing, music, fitness]"
5,event-2025-06-11-1,Tech Lunch & Learn: AI Frontiers,2025-06-11 12:00,2025-06-11 13:30,"The Digital Atrium, AgentsVille","Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI cr

## The ItineraryAgent

First we will review the Pydantic objects used for defining the output of our agent, the TravelPlan, ItineraryDay, Activity, and Weather classes.

Second, we will create a Chain-of-Thought prompt to guide the agent in planning the trip. This prompt will instruct the agent to consider the weather, activities, and user preferences when creating the itinerary.

Third, we will run the agent to produce the TravelPlan object, which will will refine in the following steps.

In [9]:
# Review the data structure we will use for representing a TravelPlan, which includes
# weather, activity_recommendations, and itinerary for each day of the trip.
# Our goal is to take a VacationInfo object and return a TravelPlan object.
# No changes are needed here.

class Weather(BaseModel):
    temperature: float
    temperature_unit: str
    condition: str


class Activity(BaseModel):
    activity_id: str
    name: str
    start_time: datetime.datetime
    end_time: datetime.datetime
    location: str
    description: str
    price: int
    related_interests: List[Interest]


class ActivityRecommendation(BaseModel):
    activity: Activity
    reasons_for_recommendation: List[str]


class ItineraryDay(BaseModel):
    date: datetime.date
    weather: Weather
    activity_recommendations: List[ActivityRecommendation]


class TravelPlan(BaseModel):
    city: str
    start_date: datetime.date
    end_date: datetime.date
    total_cost: int
    itinerary_days: List[ItineraryDay]

In [10]:
# =========================
# ItineraryAgent (FIXED)
# =========================

import json
import datetime
import pandas as pd
from typing import Optional, Any

from project_lib import ChatAgent, print_in_box
from json_repair import repair_json

# --- Helpers for putting context into the prompt ---
def to_records_json(x: Any) -> str:
    """
    Convert either a DataFrame or list-like into a JSON array of record dicts.
    """
    if hasattr(x, "to_json"):  # pandas DataFrame
        return x.to_json(orient="records", indent=2, date_format="iso")
    return json.dumps(x, indent=2, default=str)


def extract_first_json_object(text: str) -> str:
    """
    Extract the first top-level {...} JSON object from a model response,
    even if it contains extra text.
    """
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1 or end <= start:
        raise ValueError("No JSON object found in model output.")
    return text[start : end + 1]


# --- Build a STRICT schema that matches your Pydantic models exactly ---
# These models are defined earlier in your notebook:
# Weather(temperature, temperature_unit, condition)
# Activity(activity_id, name, start_time, end_time, location, description, price, related_interests)
# ActivityRecommendation(activity, reasons_for_recommendation)
# ItineraryDay(date, weather, activity_recommendations)
# TravelPlan(city, start_date, end_date, total_cost, itinerary_days)

ITINERARY_AGENT_SYSTEM_PROMPT = f"""
[Role]
You are an expert travel planning agent.

[Task]
Create a travel itinerary for the given VacationInfo, using ONLY the provided:
- Weather records
- Activity records

You MUST:
- Recommend ONLY activities that appear in the provided Activities Data.
- Use ONLY activity_id values that appear in Activities Data.
- Put activities on the correct date.
- Align recommendations with the travelers' interests.
- Keep total_cost within the provided budget.
- Ensure at least ONE recommended activity per day.
- Avoid weather-incompatible activities (e.g., outdoor-only during storms/heavy rain) based on the activity description and weather condition.
- Do NOT invent activities, IDs, prices, times, or weather.

[Important Data Rules]
- "activity_recommendations" MUST be a list of objects with EXACTLY:
  - "activity": an Activity object (NOT nested again inside another "activity")
  - "reasons_for_recommendation": list of strings
- "activity" must NOT be double-wrapped like {{"activity": {{"activity": ...}}}}.
- start_time and end_time MUST be ISO datetimes (e.g., "2025-06-10T09:00:00"), not "HH:MM".
- Use the weather fields EXACTLY as: temperature, temperature_unit, condition.

[Output Format — STRICT]
Return ONLY a single valid JSON object and NOTHING else.
No markdown. No code fences. No extra keys.

The JSON MUST match EXACTLY this schema:

{{
  "city": "<string>",
  "start_date": "<YYYY-MM-DD>",
  "end_date": "<YYYY-MM-DD>",
  "total_cost": <int>,
  "itinerary_days": [
    {{
      "date": "<YYYY-MM-DD>",
      "weather": {{
        "temperature": <number>,
        "temperature_unit": "<string>",
        "condition": "<string>"
      }},
      "activity_recommendations": [
        {{
          "activity": {{
            "activity_id": "<string>",
            "name": "<string>",
            "start_time": "<YYYY-MM-DDTHH:MM:SS>",
            "end_time": "<YYYY-MM-DDTHH:MM:SS>",
            "location": "<string>",
            "description": "<string>",
            "price": <int>,
            "related_interests": ["<string>", "..."]
          }},
          "reasons_for_recommendation": ["<string>", "..."]
        }}
      ]
    }}
  ]
}}

[Context]

## Weather Data (records)
{to_records_json(weather_for_dates_df)}

## Activities Data (records)
{to_records_json(activities_for_dates_df)}
""".strip()

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ prompt must contain a TASK section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ prompt must contain an OUTPUT FORMAT section"


class ItineraryAgent(ChatAgent):
    """Plans itineraries based on VacationInfo, Weather records, and Activities records."""
    system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT

    def get_itinerary(self, vacation_info, model: Optional[Any] = None):
        """
        Generates a TravelPlan JSON, parses it, and validates it with Pydantic TravelPlan.
        """
        # 1) Call model
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # 2) Extract and repair JSON (robust against extra text)
        json_text = extract_first_json_object(response)
        json_text = repair_json(json_text)  # fixes minor formatting issues

        # 3) Validate against your Pydantic model
        from __main__ import TravelPlan  # uses the TravelPlan class defined earlier in the notebook
        travel_plan = TravelPlan.model_validate_json(json_text)

        return travel_plan


# Instantiate the agent
itinerary_agent = ItineraryAgent(client=client, model=MODEL)



╔══════════════════════════════════════════[ ItineraryAgent - System Prompt ]══════════════════════════════════════════╗
║ [Role]                                                                                                               ║
║ You are an expert travel planning agent.                                                                             ║
║ [Task]                                                                                                               ║
║ Create a travel itinerary for the given VacationInfo, using ONLY the provided:                                       ║
║ - Weather records                                                                                                    ║
║ - Activity records                                                                                                   ║
║ You MUST:                                                                                                            ║
║ - Recommend ONLY activities t

In [11]:
# Generate the travel itinerary
# No changes needed here, though you can change the model to a different one if you want.

travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "tennis",                                                                                                    ║
║         "cooking",           

## Evaluating the Itinerary

We've successfully created an itinerary, but how do we know if it's any good?

Now we will create some evaluation functions (sometimes called evals) to help us determine the quality of the itinerary. We will not only want our final output to be of the highest quality possible initially, but we also want to give the chance for the LLM to reflect on its own output and make improvements at a second pass.

If the itinerary does not meet all the criteria specified here, no worries! Afterwards, we will implement a feedback loop that will allow the agent to improve its plan iteratively.

In [12]:
# Helper functions for running the evaluation functions
# No change needed here.

class AgentError(Exception):
    pass


class EvaluationResults(BaseModel):
    success: bool
    failures: List[str]
    eval_functions: List[str]


def get_eval_results(vacation_info, final_output, eval_functions) -> EvaluationResults:
    """
    Evaluates the final output of the itinerary agent against a set of evaluation functions.
    Args:
        vacation_info (VacationInfo): The vacation information used to generate the itinerary.
        final_output (TravelPlan): The final output from the itinerary agent.
        eval_functions (List[callable]): A list of evaluation functions to apply.
    Returns:
        EvaluationResults: An object containing the success status, any failures, and the names of the evaluation functions used.
    """
    from project_lib import print_in_box
    if not isinstance(vacation_info, VacationInfo):
        raise ValueError("vacation_info must be an instance of VacationInfo")
    if not isinstance(final_output, TravelPlan):
        raise ValueError("final_output must be an instance of TravelPlan")
    if not isinstance(eval_functions, list) or not all(
        callable(fn) for fn in eval_functions
    ):
        raise ValueError("eval_functions must be a list of callable functions")
    eval_results = []
    for eval_fn in eval_functions:
        try:
            eval_fn(vacation_info, final_output)
        except AgentError as e:
            error_msg = str(e)
            print_in_box(error_msg, title="Evaluation Error")
            print("\n\n")

            eval_results.append(error_msg)
    return EvaluationResults(
        success=len(eval_results) == 0,
        failures=eval_results,
        eval_functions=[fn.__name__ for fn in eval_functions],
    )


In [13]:
# Basic evaluation functions
# No changes needed here.

def eval_start_end_dates_match(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the arrival and departure dates in vacation_info match the start and end dates in final_output.

    Args:
        vacation_info (dict): Contains the vacation details including arrival and departure dates
        final_output (dict): Contains the itinerary details including start and end dates

    Raises:
        AgentError: If either the arrival date doesn't match the start date or the departure date doesn't match the end date
    """
    if (
        vacation_info.date_of_arrival != final_output.start_date
        or vacation_info.date_of_departure != final_output.end_date
    ):
        raise AgentError(
            f"Dates do not match: {vacation_info.date_of_arrival} != {final_output.start_date} or {vacation_info.date_of_departure} != {final_output.end_date}"
        )

    if final_output.start_date > final_output.end_date:
        raise AgentError(
            f"Start date is after end date: {final_output.start_date} > {final_output.end_date}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_start_end_dates_match],
)

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match'])

In [14]:
# Evaluation functions related to the budget and total cost
# No changes needed here.


def eval_total_cost_is_accurate(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output matches the sum of all activity prices.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including activities with prices and total cost

    Raises:
        AgentError: If the calculated total cost doesn't match the stated total cost
    """
    actual_total_cost = 0

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            actual_total_cost += activity_recommendation.activity.price

    stated_total_cost = int(final_output.total_cost)

    if actual_total_cost != stated_total_cost:
        raise AgentError(
            f"Stated total cost does not match calculated total cost: {actual_total_cost} != {stated_total_cost}"
        )
    
def eval_total_cost_is_within_budget(vacation_info: VacationInfo, final_output: TravelPlan):
    """Verifies that the total cost stated in final_output is within the budget specified in vacation_info.

    Args:
        vacation_info (dict): Contains the vacation details including budget
        final_output (dict): Contains the itinerary details including total cost

    Raises:
        AgentError: If the total cost exceeds the budget
    """
    stated_total_cost = int(final_output.total_cost)
    if stated_total_cost > vacation_info.budget:
        raise AgentError(
            f"Total cost exceeds budget: {stated_total_cost} > {vacation_info.budget}"
        )

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_total_cost_is_accurate, eval_total_cost_is_within_budget],
)



╔═════════════════════════════════════════════════[ Evaluation Error ]═════════════════════════════════════════════════╗
║ Stated total cost does not match calculated total cost: 40 != 125                                                    ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝





EvaluationResults(success=False, failures=['Stated total cost does not match calculated total cost: 40 != 125'], eval_functions=['eval_total_cost_is_accurate', 'eval_total_cost_is_within_budget'])

In [15]:
# The final output contains copies of the activities, so we need to verify that the copies are accurate
# and not hallucinated.
# No changes needed here.

def eval_itinerary_events_match_actual_events(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the events listed in the itinerary match the actual events

    Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    from project_lib import call_activity_by_id_api_mocked
    event_ids_not_matching = []
    event_ids_missing = []

    for itinerary_day in final_output.itinerary_days:
        for activity_recommendation in itinerary_day.activity_recommendations:
            event_id = activity_recommendation.activity.activity_id
            # Assuming get_event_by_id is a function that retrieves the event by its ID

            reference_event = call_activity_by_id_api_mocked(event_id)

            if reference_event is None:
                event_ids_missing.append(event_id)

            elif Activity(**reference_event) != activity_recommendation.activity:
                print(
                    "---\n"
                    f"Event ID {event_id} does not match the reference event:\n"
                    f"Reference Event: {reference_event}\n"
                    f"Activity Event: {activity_recommendation.activity.model_dump()}"
                )
                event_ids_not_matching.append(event_id)
            else:
                # The event matches, so we can continue
                pass

    if event_ids_missing or event_ids_not_matching:
        raise AgentError(
            f"Event IDs missing: {event_ids_missing}\nEvent IDs not matching: {event_ids_not_matching}"
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_events_match_actual_events],
)


---
Event ID event-2025-06-11-1 does not match the reference event:
Reference Event: {'activity_id': 'event-2025-06-11-1', 'name': 'Tech Lunch & Learn: AI Frontiers', 'start_time': '2025-06-11 12:00', 'end_time': '2025-06-11 13:30', 'location': 'The Digital Atrium, AgentsVille', 'description': "Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect with others passionate about technology, AI, and the digital world. Whether you're a seasoned developer or just curious about tech, this event is for you! Related interests: technology, music (sound tech demos), photography (AI imaging), writing (AI creativity).", 'price': 20, 'related_interests': ['technology', 'music', 'photography', 'writing']}
Activity Event: {'

EvaluationResults(success=False, failures=["Event IDs missing: []\nEvent IDs not matching: ['event-2025-06-11-1']"], eval_functions=['eval_itinerary_events_match_actual_events'])

In [16]:
# Check that the itinerary includes at least one activity matching each traveler's interests.
# No changes needed here.

def eval_itinerary_satisfies_interests(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that the itinerary includes activities matching each traveler's interests.

    This function checks that each traveler has at least one activity in the itinerary that matches their interests.

        Args:
        vacation_info (dict): Contains the vacation details including traveler information and their interests
        final_output (dict): Contains the itinerary details including daily activities

    Raises:
        AgentError: If any traveler has no matching activities or if one traveler has more than twice
                   the number of matching activities compared to another traveler
    """
    traveler_to_interests = {}
    traveler_to_interest_hit_counts = {}

    for traveler in vacation_info.travelers:
        traveler_to_interests[traveler.name] = traveler.interests
        traveler_to_interest_hit_counts[traveler.name] = 0

    for traveler_name, interests in traveler_to_interests.items():
        for itinerary_day in final_output.itinerary_days:
            for activity_recommendation in itinerary_day.activity_recommendations:
                # Check if the activity matches any of the traveler's interests
                matching_interests = set(traveler_to_interests[traveler_name]) & set(
                    activity_recommendation.activity.related_interests
                )

                if matching_interests:
                    traveler_to_interest_hit_counts[traveler_name] += 1
                    print(
                        f"✅ Traveler {traveler_name} has a match with interest {matching_interests} at {activity_recommendation.activity.name}"
                    )

    travelers_with_no_interest_hits = [
        traveler
        for traveler, interest_hit_count in traveler_to_interest_hit_counts.items()
        if interest_hit_count == 0
    ]

    # If any of the travelers have 0 matches, raise an error
    if travelers_with_no_interest_hits:
        raise AgentError(
            f"Travelers {travelers_with_no_interest_hits} has no matches with the itinerary."
        )


get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)


✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {technology} at Tech Lunch & Learn: AI Frontiers
✅ Traveler Hiro has a match with interest {music} at Tech Lunch & Learn: AI Frontiers


EvaluationResults(success=True, failures=[], eval_functions=['eval_itinerary_satisfies_interests'])

In [17]:
# ==========================================
# Weather compatibility eval (FIXED)
# ==========================================

ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
[Role]: Weather Compatibility Evaluation Agent

## Task
Given an activity and a weather condition, decide whether the activity is compatible.

Rules:
- If there is NOT enough information, assume IS_COMPATIBLE.
- If the activity has an indoor alternative or backup option in its description, assume IS_COMPATIBLE unless clearly impossible.
- Outdoor-only activities should be marked IS_INCOMPATIBLE during heavy rain, thunderstorms/storms, snow, blizzards, or other extreme weather.
- If the weather condition is mild (sunny, cloudy, light breeze, etc.), assume IS_COMPATIBLE unless the activity description clearly contradicts it.

## Output format (STRICT)
Return exactly ONE line with only one of:
IS_COMPATIBLE
IS_INCOMPATIBLE
""".strip()


def _parse_compatibility_label(resp: str) -> bool:
    """
    Returns True if compatible, False if incompatible.
    Raises if the response isn't one of the expected labels.
    """
    if resp is None:
        raise RuntimeError("Model returned None response.")
    label = resp.strip().upper()

    # Must check INCOMPATIBLE first if using substring logic, but we avoid substring logic altogether:
    if label == "IS_INCOMPATIBLE":
        return False
    if label == "IS_COMPATIBLE":
        return True

    # Sometimes models add extra text; fall back to searching for exact tokens on lines
    lines = [ln.strip().upper() for ln in resp.splitlines() if ln.strip()]
    if "IS_INCOMPATIBLE" in lines:
        return False
    if "IS_COMPATIBLE" in lines:
        return True

    raise RuntimeError(
        f"Unexpected response from model. Expected exactly IS_COMPATIBLE or IS_INCOMPATIBLE.\nGot:\n{resp}"
    )


def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """
    Verifies that no outdoor-only activities are scheduled during inclement weather,
    using an LLM classifier over (activity description, weather condition).
    """
    from project_lib import do_chat_completion

    incompatible = []  # store tuples for better debugging

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for rec in itinerary_day.activity_recommendations:
            activity = rec.activity

            user_payload = (
                f"Activity: {activity.name}\n"
                f"Description: {activity.description}\n"
                f"Weather Condition: {weather_condition}\n"
            )

            resp = do_chat_completion(
                messages=[
                    {"role": "system", "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT},
                    {"role": "user", "content": user_payload},
                ],
                client=client,
                model=OpenAIModel.GPT_41_NANO,  # fast + cheap for repeated calls
            )

            is_compatible = _parse_compatibility_label(resp)

            if is_compatible:
                print(
                    f"✅ Compatible: {activity.name} on {itinerary_day.date} (weather: '{weather_condition}')"
                )
            else:
                print(
                    f"❌ Incompatible: {activity.name} on {itinerary_day.date} (weather: '{weather_condition}')"
                )
                incompatible.append((str(itinerary_day.date), activity.name, weather_condition))

    if incompatible:
        details = "\n".join([f"- {d}: {name} (weather: {w})" for d, name, w in incompatible])
        raise AgentError(
            "Activities that may be ruined by inclement weather:\n" + details
        )


# Run just this eval
eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_activities_and_weather_are_compatible],
)

eval_results


✅ Compatible: FutureTech Breakfast Meet-Up on 2025-06-10 (weather: 'clear')
✅ Compatible: Tech Lunch & Learn: AI Frontiers on 2025-06-11 (weather: 'partly cloudy')


EvaluationResults(success=True, failures=[], eval_functions=['eval_activities_and_weather_are_compatible'])

In [18]:
# Run all of the evaluation functions again
# No changes needed here.

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
]

eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

eval_results.model_dump()


╔═════════════════════════════════════════════════[ Evaluation Error ]═════════════════════════════════════════════════╗
║ Stated total cost does not match calculated total cost: 40 != 125                                                    ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝



---
Event ID event-2025-06-11-1 does not match the reference event:
Reference Event: {'activity_id': 'event-2025-06-11-1', 'name': 'Tech Lunch & Learn: AI Frontiers', 'start_time': '2025-06-11 12:00', 'end_time': '2025-06-11 13:30', 'location': 'The Digital Atrium, AgentsVille', 'description': "Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect w

{'success': False,
 'failures': ['Stated total cost does not match calculated total cost: 40 != 125',
  "Event IDs missing: []\nEvent IDs not matching: ['event-2025-06-11-1']"],
 'eval_functions': ['eval_start_end_dates_match',
  'eval_total_cost_is_accurate',
  'eval_itinerary_events_match_actual_events',
  'eval_itinerary_satisfies_interests',
  'eval_total_cost_is_within_budget',
  'eval_activities_and_weather_are_compatible']}

## Defining the Tools

Our ItineraryRevisionAgent will be a ReAct-based agent that will use tools to:
- Evaluate/Re-evaluate the itinerary
- Use a calculator since LLMs sometimes struggle with arithmetic
- Call the activities API to get more information about activities
- Return the final itinerary


In [19]:
# Helper function to generate tool descriptions from function docstrings
# No changes needed here.

def get_tool_descriptions_string(fns):
    """Generates a tool description from a function's docstring.
    Args:
        fns (list): List of functions to generate descriptions for.
    Returns:
        str: A formatted string containing the function names and their descriptions."""
    resp = ""
    for fn in fns:
        function_name = fn.__name__
        function_doc = fn.__doc__ or "No description provided."

        resp += f"* `{function_name}`: {function_doc}\n"

    return resp

In [20]:
# Define the calculator tool that evaluates mathematical expressions.
# No changes needed here.

def calculator_tool(input_expression) -> float:
    """Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    """
    import numexpr as ne
    return float(ne.evaluate(input_expression))


assert calculator_tool("1 + 1") == 2.0

print(get_tool_descriptions_string([calculator_tool]))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    



In [21]:
from typing import List
import datetime

def get_activities_by_date_tool(date: str, city: str) -> List[dict]:
    """
    Fetch available activities for a given date and city.

    - Calls the mocked activities API.
    - Validates each returned object against the Activity Pydantic model.
    - Returns a list of dictionaries that match the Activity schema, with datetimes serialized to ISO strings.

    Args:
        date: 'YYYY-MM-DD'
        city: city name

    Returns:
        List[dict]: Validated activities (Activity schema) as JSON-serializable dicts.
    """
    from project_lib import call_activities_api_mocked

    # Validate date input early (helps the ReAct agent + prevents silent bugs)
    try:
        requested_date = datetime.date.fromisoformat(date)
    except ValueError:
        raise ValueError("date must be in 'YYYY-MM-DD' format")

    resp = call_activities_api_mocked(date=date, city=city)

    validated: List[dict] = []
    for item in resp:
        act = Activity.model_validate(item)

        # Safety: ensure activity actually belongs to requested date
        # (API should already do this, but it prevents leakage if the mock changes)
        if act.start_time.date() != requested_date:
            continue

        validated.append(
            act.model_dump(mode="json")  # converts datetimes -> ISO strings
        )

    return validated


assert len(get_activities_by_date_tool("2025-06-10", "AgentsVille")) > 0
print(get_tool_descriptions_string([get_activities_by_date_tool]))

* `get_activities_by_date_tool`: 
    Fetch available activities for a given date and city.

    - Calls the mocked activities API.
    - Validates each returned object against the Activity Pydantic model.
    - Returns a list of dictionaries that match the Activity schema, with datetimes serialized to ISO strings.

    Args:
        date: 'YYYY-MM-DD'
        city: city name

    Returns:
        List[dict]: Validated activities (Activity schema) as JSON-serializable dicts.
    



In [22]:
# Tool to run all evaluation functions on a travel plan.
# No changes needed here.

def run_evals_tool(travel_plan: TravelPlan) -> dict:
    """Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    """
    if isinstance(travel_plan, dict):
        travel_plan = TravelPlan.model_validate(travel_plan)

    resp = get_eval_results(
        vacation_info=vacation_info,
        final_output=travel_plan,
        eval_functions=ALL_EVAL_FUNCTIONS,
    )
    return {
        # Show the success status and any failures
        "success": resp.success,
        "failures": resp.failures,
    }


print(get_tool_descriptions_string([run_evals_tool]))

* `run_evals_tool`: Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    



In [23]:
# Let's double check that the tool works as expected.
# You should see the same results as before
run_evals_tool(travel_plan=travel_plan_1)


╔═════════════════════════════════════════════════[ Evaluation Error ]═════════════════════════════════════════════════╗
║ Stated total cost does not match calculated total cost: 40 != 125                                                    ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝



---
Event ID event-2025-06-11-1 does not match the reference event:
Reference Event: {'activity_id': 'event-2025-06-11-1', 'name': 'Tech Lunch & Learn: AI Frontiers', 'start_time': '2025-06-11 12:00', 'end_time': '2025-06-11 13:30', 'location': 'The Digital Atrium, AgentsVille', 'description': "Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect w

{'success': False,
 'failures': ['Stated total cost does not match calculated total cost: 40 != 125',
  "Event IDs missing: []\nEvent IDs not matching: ['event-2025-06-11-1']"]}

In [24]:
# A tool to return the final travel plan
# No changes needed here.

def final_answer_tool(final_output: TravelPlan) -> TravelPlan:
    """Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    """
    return final_output


print(get_tool_descriptions_string([final_answer_tool]))

* `final_answer_tool`: Returns the final travel plan

    Args:
        final_output (TravelPlan): The final travel plan to return.

    Returns:
        TravelPlan: The final travel plan.
    



In [25]:
# List of all tools available for the agent
# No changes needed here.

ALL_TOOLS = [
    calculator_tool,
    get_activities_by_date_tool,
    run_evals_tool,
    final_answer_tool,
]
print(get_tool_descriptions_string(ALL_TOOLS))

* `calculator_tool`: Evaluates a mathematical expression and returns the result as a float.

    Args:
        input_expression (str): A string containing a valid mathematical expression to evaluate.

    Returns:
        float: The result of the evaluated expression.

    Example:
        >>> calculator_tool("1 + 1")
        2.0
    
* `get_activities_by_date_tool`: 
    Fetch available activities for a given date and city.

    - Calls the mocked activities API.
    - Validates each returned object against the Activity Pydantic model.
    - Returns a list of dictionaries that match the Activity schema, with datetimes serialized to ISO strings.

    Args:
        date: 'YYYY-MM-DD'
        city: city name

    Returns:
        List[dict]: Validated activities (Activity schema) as JSON-serializable dicts.
    
* `run_evals_tool`: Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Return

## The ItineraryRevisionAgent

The ItineraryRevisionAgent will
* take initial feedback from the user about the itinerary and
* use the tools defined above

to refine original itinerary iteratively using a ReAct-based approach.

In [26]:
# Get the traveler's feedback and create a new evaluation function to check if the feedback was incorporated.
# No changes needed here.

TRAVELER_FEEDBACK = "I want to have at least two activities per day."


def eval_traveler_feedback_is_incorporated(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Checks if the traveler's feedback was incorporated into the revised travel plan.

    Args:
        vacation_info (VacationInfo): The vacation information.
        final_output (TravelPlan): The revised travel plan.

    Raises:
        AgentError: If the traveler's feedback was not successfully incorporated.
    """

    agent = ChatAgent(
        system_prompt="""You are an expert in evaluating whether a travel plan incorporates traveler feedback.

    ## Output Format

    Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

        ANALYSIS:
        * [step-by-step analysis]


        FINAL OUTPUT:
        [FULLY_INCORPORATED, PARTIALLY_INCORPORATED, NOT_INCORPORATED, or UNKNOWN]
        REASON: [reasoning for the final output]

    """,
        client=client,
        model=OpenAIModel.GPT_41,  # Use a powerful model for checking traveler feedback
    )

    resp = agent.chat(
        f"""Traveler Feedback: {TRAVELER_FEEDBACK}
    Revised Travel Plan: {final_output.model_dump_json()}
    """,
    )
    if "FINAL OUTPUT:" not in resp:
        raise RuntimeError(
            f"Unexpected response from the model: {resp}. Expected 'FINAL OUTPUT:'."
        )
    if "FULLY_INCORPORATED" not in resp:
        final_output = resp.split("FINAL OUTPUT:")[-1].strip()
        raise AgentError(
            f"Traveler feedback was not successfully incorporated into the revised travel plan. Response: {final_output}"
        )

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
    eval_traveler_feedback_is_incorporated,  # Add this new evaluation
]

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)


╔═════════════════════════════════════════════════[ Evaluation Error ]═════════════════════════════════════════════════╗
║ Stated total cost does not match calculated total cost: 40 != 125                                                    ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╝



---
Event ID event-2025-06-11-1 does not match the reference event:
Reference Event: {'activity_id': 'event-2025-06-11-1', 'name': 'Tech Lunch & Learn: AI Frontiers', 'start_time': '2025-06-11 12:00', 'end_time': '2025-06-11 13:30', 'location': 'The Digital Atrium, AgentsVille', 'description': "Join fellow tech enthusiasts for a dynamic lunchtime event exploring the future of artificial intelligence! Held indoors at The Digital Atrium, this Tech Lunch & Learn features engaging lightning talks, interactive demos, and networking opportunities all centered around technology and innovation. Enjoy light lunch fare as you connect w

EvaluationResults(success=False, failures=['Stated total cost does not match calculated total cost: 40 != 125', "Event IDs missing: []\nEvent IDs not matching: ['event-2025-06-11-1']", 'Traveler feedback was not successfully incorporated into the revised travel plan. Response: NOT_INCORPORATED\nREASON: The revised travel plan provides only one activity per day, whereas the traveler requested at least two activities per day. The feedback has not been addressed.'], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [31]:
# ==========================================
# ItineraryRevisionAgent (ACCURATE + ROBUST)
# ==========================================

from project_lib import print_in_box, ChatAgent
from typing import Optional
import json

# --- System prompt stays STATIC (no f-string) ---
ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = """
[Role]
You are an Itinerary Revision Agent.

[Goal]
Revise an existing TravelPlan to fully incorporate traveler feedback while satisfying ALL constraints
and passing ALL evaluation checks.

[Hard Constraints — MUST NOT BE VIOLATED]
- All activities MUST be real and MUST come from the mocked activities API.
- You may ONLY obtain activities via get_activities_by_date_tool.
- When you include an activity, you MUST copy it EXACTLY as returned by the tool
  (no edits to id, name, times, description, price, or related_interests).
- The final plan MUST contain at least TWO activity_recommendations for EVERY itinerary day.
- total_cost MUST equal the sum of prices of ALL recommended activities and MUST be within budget.

[Available Tools — EXACT NAMES ONLY]
- calculator_tool
- get_activities_by_date_tool
- run_evals_tool
- final_answer_tool

[Output Format — STRICT]
THOUGHT:
<short reasoning>

ACTION:
{
  "tool_name": "...",
  "arguments": { ... }
}
""".strip()


# --- Normalization helper (unchanged) ---
def normalize_travel_plan_dict(tp: dict) -> dict:
    if not isinstance(tp, dict):
        return tp

    for day in tp.get("itinerary_days", []):
        recs = day.get("activity_recommendations", [])
        if isinstance(recs, dict):
            recs = recs.get("activity_recommendations", [])
        for rec in recs:
            act = rec.get("activity")
            while isinstance(act, dict) and "activity" in act:
                act = act["activity"]
            rec["activity"] = act
            if not isinstance(rec.get("reasons_for_recommendation"), list):
                rec["reasons_for_recommendation"] = ["Matches traveler preferences."]
        day["activity_recommendations"] = recs

    return tp


# ==========================================
# Agent Class (REWRITTEN + HARDENED)
# ==========================================

class ItineraryRevisionAgent(ChatAgent):
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT
    tools = ALL_TOOLS

    def get_observation_string(self, tool_call_obj) -> str:
        """Execute a tool call and wrap its result as an OBSERVATION string."""
        if not isinstance(tool_call_obj, dict):
            return f"OBSERVATION: Tool call must be a dict, got {type(tool_call_obj)}."

        tool_name = tool_call_obj.get("tool_name")
        arguments = tool_call_obj.get("arguments", {})

        # tool_name validation
        if not tool_name or not isinstance(tool_name, str):
            return "OBSERVATION: Missing or invalid tool_name."

        # ✅ Harden arguments to dict
        if arguments is None:
            arguments = {}
        if not isinstance(arguments, dict):
            return (
                "OBSERVATION: Invalid arguments type. Expected a JSON object (dict). "
                f"Got {type(arguments)}. Please resend ACTION with arguments as an object."
            )

        # resolve tool
        tool_fn = next((t for t in self.tools if t.__name__ == tool_name), None)
        if tool_fn is None:
            return (
                f"OBSERVATION: Unknown tool '{tool_name}'. "
                f"Available: {[t.__name__ for t in self.tools]}"
            )

        # Normalize payloads before evals
        if tool_name == "run_evals_tool":
            # Some models pass the plan under different keys; normalize here
            if isinstance(arguments.get("travel_plan"), dict):
                arguments["travel_plan"] = normalize_travel_plan_dict(arguments["travel_plan"])
            elif isinstance(arguments.get("final_output"), dict):
                arguments["travel_plan"] = normalize_travel_plan_dict(arguments["final_output"])
                arguments.pop("final_output", None)

        try:
            tool_response = tool_fn(**arguments)
            return f"OBSERVATION: {tool_name} response:\n{tool_response}"
        except Exception as e:
            return f"OBSERVATION: Error calling {tool_name}: {e}"

    def _extract_final_travel_plan_candidate(self, tool_call: dict) -> dict | None:
        """
        Accept multiple common payload shapes for final_answer_tool:
        1) {"arguments": {"final_output": {...}}}
        2) {"arguments": {"travel_plan": {...}}}
        3) {"arguments": {...TravelPlan fields...}}
        """
        args = tool_call.get("arguments", {})
        if args is None:
            args = {}

        # If args isn't a dict, we can't validate it
        if not isinstance(args, dict):
            return None

        candidate = args.get("final_output") or args.get("travel_plan") or args

        if not isinstance(candidate, dict) or not candidate:
            return None

        return normalize_travel_plan_dict(candidate)

    def run_react_cycle(
        self,
        original_travel_plan: TravelPlan,
        max_steps: int = 30,
        model: Optional[OpenAIModel] = None,
        client=None,
    ) -> TravelPlan:
        """Run a ReAct loop until final_answer_tool is called or max_steps is reached."""
        from json_repair import repair_json

        # --- Inject itinerary ---
        self.add_message(
            role="user",
            content=f"Here is the itinerary for revision (JSON):\n{original_travel_plan.model_dump_json()}",
        )

        # ✅ Inject traveler feedback SAFELY (NO f-string in prompt)
        self.add_message(
            role="user",
            content=f"Traveler feedback (MUST incorporate): {TRAVELER_FEEDBACK}",
        )

        # ✅ Force correct first step
        self.add_message(
            role="user",
            content="IMPORTANT: First ACTION must be run_evals_tool on the current plan.",
        )

        last_resp = ""
        for _ in range(max_steps):
            last_resp = self.get_response(model=model, client=client) or ""

            if "ACTION:" not in last_resp:
                self.add_message(
                    role="user",
                    content="OBSERVATION: Missing ACTION. Reply with THOUGHT and one ACTION JSON.",
                )
                continue

            action_json = repair_json(last_resp.split("ACTION:", 1)[1].strip())

            try:
                tool_call = json.loads(action_json)
            except Exception:
                self.add_message(
                    role="user",
                    content=f"OBSERVATION: Invalid ACTION JSON. Must be a JSON object.\n{action_json}",
                )
                continue

            tool_name = tool_call.get("tool_name")

            # ✅ Robust finalization (handles missing/alternate keys)
            if tool_name == "final_answer_tool":
                candidate = self._extract_final_travel_plan_candidate(tool_call)
                if candidate is None:
                    self.add_message(
                        role="user",
                        content=(
                            "OBSERVATION: final_answer_tool was called without a valid TravelPlan. "
                            "Call final_answer_tool again with arguments containing the FULL TravelPlan JSON as "
                            "{'final_output': <TravelPlan>}."
                        ),
                    )
                    continue

                try:
                    return TravelPlan.model_validate(candidate)
                except Exception as e:
                    self.add_message(
                        role="user",
                        content=(
                            "OBSERVATION: The TravelPlan passed to final_answer_tool is invalid. "
                            f"Validation error: {e}. Please call final_answer_tool again with a FULL valid TravelPlan."
                        ),
                    )
                    continue

            # Non-final tools
            observation = self.get_observation_string(tool_call)
            self.add_message(role="user", content=observation)

        raise RuntimeError(
            f"ReAct cycle did not converge within max_steps={max_steps}. Last response:\n{last_resp}"
        )


# ==========================================
# Instantiate + smoke test
# ==========================================

itinerary_revision_agent = ItineraryRevisionAgent()

resp = itinerary_revision_agent.chat(
    user_message=f"Here is the itinerary for revision:\n{travel_plan_1.model_dump_json(indent=2)}",
    add_to_messages=False,
    model=MODEL,
    client=client,
) or ""

print_in_box(resp, "Raw Response")



╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ [Role]                                                                                                               ║
║ You are an Itinerary Revision Agent.                                                                                 ║
║ [Goal]                                                                                                               ║
║ Revise an existing TravelPlan to fully incorporate traveler feedback while satisfying ALL constraints                ║
║ and passing ALL evaluation checks.                                                                                   ║
║ [Hard Constraints — MUST NOT BE VIOLATED]                                                                            ║
║ - All activities MUST be real and MUST come from the mocked activities API.                                          ║
║ - You may ONLY obtain activit

In [32]:
# Generate the travel itinerary
# No changes needed here, though you can change the model to a different one if you want.

travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")


╔═══════════════════════════════════════════[ ItineraryAgent - User Prompt ]═══════════════════════════════════════════╗
║ {                                                                                                                    ║
║   "travelers": [                                                                                                     ║
║     {                                                                                                                ║
║       "name": "Yuri",                                                                                                ║
║       "age": 30,                                                                                                     ║
║       "interests": [                                                                                                 ║
║         "tennis",                                                                                                    ║
║         "cooking",           

In [33]:
itinerary_revision_agent = ItineraryRevisionAgent()

travel_plan_2 = itinerary_revision_agent.run_react_cycle(
    original_travel_plan=travel_plan_1,
    max_steps=30,
    model=MODEL,
    client=client,
)

print("✅ Revised itinerary generated successfully.")


╔══════════════════════════════════════[ ItineraryRevisionAgent - System Prompt ]══════════════════════════════════════╗
║ [Role]                                                                                                               ║
║ You are an Itinerary Revision Agent.                                                                                 ║
║ [Goal]                                                                                                               ║
║ Revise an existing TravelPlan to fully incorporate traveler feedback while satisfying ALL constraints                ║
║ and passing ALL evaluation checks.                                                                                   ║
║ [Hard Constraints — MUST NOT BE VIOLATED]                                                                            ║
║ - All activities MUST be real and MUST come from the mocked activities API.                                          ║
║ - You may ONLY obtain activit

In [34]:
# Last let's double check that the revised travel plan passes all evaluation functions.
# No changes needed here.

eval_results_2 = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_2,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

assert eval_results_2.success, f"❌ Read the traces above and modify the system prompt.\n\nFailures: {eval_results_2.failures}"

print("✅ All evaluation functions passed successfully for the revised travel plan.")

eval_results_2

✅ Traveler Yuri has a match with interest {technology} at FutureTech Breakfast Meet-Up
✅ Traveler Yuri has a match with interest {cooking} at Palette & Palate: Art Meets Cooking Experience
✅ Traveler Hiro has a match with interest {art} at Artful Athletics: Paint & Play Extravaganza
✅ Traveler Hiro has a match with interest {music, art} at AgentsVille Art & Music Fusion Fest
✅ Traveler Hiro has a match with interest {art} at Palette & Palate: Art Meets Cooking Experience
✅ Compatible: FutureTech Breakfast Meet-Up on 2025-06-10 (weather: 'clear')
✅ Compatible: Artful Athletics: Paint & Play Extravaganza on 2025-06-10 (weather: 'clear')
✅ Compatible: AgentsVille Art & Music Fusion Fest on 2025-06-11 (weather: 'partly cloudy')
✅ Compatible: Palette & Palate: Art Meets Cooking Experience on 2025-06-11 (weather: 'partly cloudy')

╔════════════════════════════════════════════[ ChatAgent - System Prompt ]═════════════════════════════════════════════╗
║ You are an expert in evaluating whether 

EvaluationResults(success=True, failures=[], eval_functions=['eval_start_end_dates_match', 'eval_total_cost_is_accurate', 'eval_itinerary_events_match_actual_events', 'eval_itinerary_satisfies_interests', 'eval_total_cost_is_within_budget', 'eval_activities_and_weather_are_compatible', 'eval_traveler_feedback_is_incorporated'])

In [35]:
# Show the final travel plan in a readable format.
# No changes needed here.

from IPython.display import display

for itinerary_day in travel_plan_2.itinerary_days:
    print(f"Date: {itinerary_day.date}")
    print(
        f"Weather: {itinerary_day.weather.condition} ({itinerary_day.weather.temperature}°{itinerary_day.weather.temperature_unit})"
    )

    activities_df = pd.DataFrame(
        [
            activity_recommendation.activity.model_dump()
            for activity_recommendation in itinerary_day.activity_recommendations
        ]
    )
    display(activities_df)

Date: 2025-06-10
Weather: clear (31.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-10-0,FutureTech Breakfast Meet-Up,2025-06-10 09:00:00,2025-06-10 11:00:00,"The Innovation Atrium, Tech District, AgentsVille","Join fellow technology enthusiasts for a dynamic morning at the FutureTech Breakfast Meet-Up! Dive into the latest trends in tech, gadget demos, and networking opportunities over coffee and fresh pastries. Held indoors at the spacious Innovation Atrium, this event is perfect for tech lovers eager to exchange ideas and discover new possibilities in a comfortable, modern setting.",20,[technology]
1,event-2025-06-10-2,Artful Athletics: Paint & Play Extravaganza,2025-06-10 15:00:00,2025-06-10 17:00:00,"Creative Courts Park, AgentsVille","Join us for an exciting afternoon at Creative Courts Park, where the worlds of art and sports collide! At 'Artful Athletics: Paint & Play Extravaganza', you'll participate in collaborative outdoor murals inspired by your favorite sports, and then get moving with fun, friendly sports mini-games. Whether you love painting or playing, this event celebrates creativity, teamwork, and the joy of movement under the open sky. Perfect for art lovers and sports enthusiasts alike—come ready to express yourself and get active! (Event is held outdoors; in case of rain, we move indoors to the Community Gym nearby.)",15,"[art, sports]"


Date: 2025-06-11
Weather: partly cloudy (34.0°celsius)


,activity_id,name,start_time,end_time,location,description,price,related_interests
0,event-2025-06-11-2,AgentsVille Art & Music Fusion Fest,2025-06-11 15:00:00,2025-06-11 17:30:00,"The Echo Gardens Amphitheater, AgentsVille","Immerse yourself in an unforgettable afternoon at the Echo Gardens Amphitheater, where the vibrant worlds of art and music collide! Surrounded by lush gardens under the open sky, enjoy live performances from talented local musicians while exploring an interactive outdoor art gallery featuring works from AgentsVille's creative community. This engaging outdoor event is perfect for art and music enthusiasts who love to be inspired and connect with fellow creatives. Don't miss out on the fusion of melodies and colors in a relaxing, friendly atmosphere!",18,"[art, music]"
1,event-2025-06-11-3,Palette & Palate: Art Meets Cooking Experience,2025-06-11 18:30:00,2025-06-11 20:30:00,"The Creative Canvas Studio, Artisanal Lane, AgentsVille","Immerse yourself in a colorful evening where art and cooking blend together! At 'Palette & Palate,' participants will begin indoors at The Creative Canvas Studio with a guided session to paint their own culinary-inspired masterpiece. Afterwards, a local chef will lead an interactive cooking class, teaching you how to craft vibrant, edible works of art. Whether you're an art enthusiast, a food lover, or both, this creative night is perfect for socializing and expressing yourself through color and flavor! All materials and ingredients are provided. This event is held indoors and welcomes all experience levels in art and cooking.",25,"[art, cooking]"


## And, just for fun!

In [36]:
# And finally, just for fun, let's narrate the trip.
# No changes needed here.

from project_lib import narrate_my_trip

narrate_my_trip(
    vacation_info=vacation_info,
    itinerary=travel_plan_2,
    client=client,
    model=MODEL,  # Optionally, you can change the model here
)


This trip involves two travelers: Yuri, a 30-year-old with interests in tennis, cooking, comedy, and technology; and Hiro, a 25-year-old who enjoys reading, music, theatre, and art. The journey is planned for the destination 'AgentsVille' from June 10 to June 12, 2025, with a total budget of 130 units.

The itinerary spans three days of engaging activities tailored to their interests, with accommodations within budget.

**Day 1 (June 10, 2025):**  
The trip begins with clear weather, offering perfect conditions for indoor activities. The first recommended activity is the FutureTech Breakfast Meet-Up held indoors at The Innovation Atrium. This event provides an excellent opportunity for technology enthusiasts to network, explore the latest tech trends, and enjoy demos over coffee and pastries—especially appealing to Yuri's interest in technology.  
In the afternoon, there's an outdoor activity at Creative Courts Park called Artful Athletics: Paint & Play Extravaganza. Participants can collaborate on outdoor murals inspired by sports and engage in sporty mini-games, celebrating art and movement—ideal for Hiro's interests in art and sports. The outdoor setting allows for a dynamic experience, with a contingency to move indoors if the weather turns rainy.

**Day 2 (June 11, 2025):**  
The weather is partly cloudy, making outdoor activities still enjoyable. In the afternoon, attendees can partake in the AgentsVille Art & Music Fusion Fest at The Echo Gardens Amphitheater. This outdoor event combines live music performances with an interactive art gallery, inspiring creativity and providing cultural enrichment, aligned with Hiro's interests in art and music.  
Later in the evening, the 'Palette & Palate' art and cooking experience takes place indoors at The Creative Canvas Studio. Participants will first create culinary-inspired artwork, then learn to cook vibrant dishes from a local chef. This creative session caters to both interests, offering a hands-on, social night in a comfortable indoor environment.

**Day 3 (June 12, 2025):**  
While not detailed in the provided information, it’s the concluding day of the trip, with departure scheduled, allowing travelers to reflect on their enriching experiences in AgentsVille.

Overall, this itinerary thoughtfully integrates indoor and outdoor activities suited to the travelers' diverse interests, all while respecting the budget constraints.

## CONGRATULATIONS! 🎉🥳👏

You have successfully planned a stellar vacation to AgentsVille! Your AI travel agent has demonstrated advanced reasoning techniques, including role-based prompting, chain-of-thought reasoning, ReAct prompting, and feedback loops

Give yourself a pat on the back for completing this project and completing this course!